# Working with Result Data

This notebook shows you how to:
- Access result data from simulation tools
- Work with output curves and arrays
- Save data to files
- Export data for analysis in other tools

## Key Concepts

nanoHUB simulation results can include:
- **Scalar values**: Single numbers (e.g., temperature, voltage)
- **Curve data**: Arrays of x-y pairs for plotting
- **Metadata**: SQUID, revision, date, etc.

This example uses `st4pnjunction`, a simulation tool (simtool=True) with rich output data.

## Setup

In [ ]:
# Add parent directory to path for local development
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

In [ ]:
from nanohubremote import Session
from nanohubresults import Results
import json

# Initialize session
auth_data = {
    "grant_type": "personal_token",
    "token": "YOUR_TOKEN_HERE"
}
session = Session(auth_data, url="https://nanohub.org/api")
results = Results(session)

print("✓ Connected to nanoHUB API")

## Fetching Sample Data

Let's fetch a result from st4pnjunction that includes curve data.

In [ ]:
print("Fetching sample result from st4pnjunction (simulation tool)...")

# IMPORTANT: The API requires .select() to specify which fields to return
query = results.query("st4pnjunction", simtool=True) \
    .filter("input.temperature", ">", 0) \
    .select("input.temperature", "input.p_len", "output.IV Characteristic", "output.Total Current") \
    .limit(1)

response = query.execute()

if response.get('results') and len(response['results']) > 0:
    sample_result = response['results'][0]
    squid = sample_result['squid']
    print(f"\n✓ Retrieved result: {squid}")
    
    # Display available fields
    print(f"\nAvailable fields in result:")
    for key in sorted(sample_result.keys()):
        if key != 'squid':
            value = sample_result[key]
            if isinstance(value, dict) and 'xaxis' in value:
                print(f"  • {key}: curve data ({len(value['xaxis'])} points)")
            else:
                print(f"  • {key}: {value}")
else:
    print("No results found!")
    print("Note: The API requires .select() to specify which fields to return")
    sample_result = None

## Understanding Curve Data

Result data often includes curve data, which is structured as dictionaries with 'xaxis' and 'yaxis' arrays:
```python
{
    'xaxis': [x1, x2, x3, ...],
    'yaxis': [y1, y2, y3, ...]
}
```

Let's examine the curve data in our result.

In [ ]:
if sample_result:
    # Find curve fields
    curve_fields = [k for k, v in sample_result.items() 
                    if isinstance(v, dict) and 'xaxis' in v]
    
    if curve_fields:
        print(f"Found {len(curve_fields)} curve field(s):\n")
        
        for field_name in curve_fields[:2]:  # Show first 2 curves
            curve = sample_result[field_name]
            print(f"{field_name}:")
            print(f"  X-axis points: {len(curve.get('xaxis', []))}")
            print(f"  Y-axis points: {len(curve.get('yaxis', []))}")
            print(f"  First few X values: {curve.get('xaxis', [])[:3]}")
            print(f"  First few Y values: {curve.get('yaxis', [])[:3]}\n")
    else:
        print("No curve data found in this result")

## Downloading Output Data

You can download specific output fields using the `download()` method. This is useful for getting the full data for curve outputs or other large datasets.

In [ ]:
if sample_result:
    # Download a specific output field
    print("Downloading output field data...")
    
    try:
        # Download the IV Characteristic curve data
        download_response = results.download(
            tool="st4pnjunction",
            squid=squid,
            field="output.IV Characteristic",
            simtool=True
        )
        
        print(f"✓ Download successful!")
        print(f"  Response type: {type(download_response)}")
        
        if isinstance(download_response, dict):
            # The download method returns the curve data
            if 'function' in download_response:
                print(f"  Data points: {len(download_response['function'])}")
                print(f"  First few values: {download_response['function'][:5]}")
            
            # Save the downloaded data
            output_filename = "iv_characteristic_data.json"
            with open(output_filename, 'w') as f:
                json.dump(download_response, f, indent=2)
            print(f"\n  Saved to: {output_filename}")
            
    except Exception as e:
        print(f"✗ Download failed: {e}")
else:
    print("No result available to download from")

## Saving Complete Results to JSON

You can easily save result data to JSON files for later analysis or sharing.

In [ ]:
if sample_result:
    # Save complete result
    output_file = "st4pnjunction_result.json"
    
    with open(output_file, "w") as f:
        json.dump(sample_result, f, indent=2)
    
    print(f"✓ Complete result saved to: {output_file}")

## Converting Curve Data to CSV

For curve data, CSV format is often more convenient for analysis tools like Excel.

In [ ]:
import csv

if sample_result:
    # Find first curve field
    curve_fields = [k for k, v in sample_result.items() 
                    if isinstance(v, dict) and 'xaxis' in v]
    
    if curve_fields:
        field_name = curve_fields[0]
        curve = sample_result[field_name]
        csv_file = f"st4pnjunction_{field_name.replace('.', '_').replace(' ', '_')}.csv"
        
        with open(csv_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['X', 'Y'])  # Header
            
            # Write data rows
            for x, y in zip(curve['xaxis'], curve['yaxis']):
                writer.writerow([x, y])
        
        print(f"✓ Curve data saved to CSV: {csv_file}")
        print(f"  Format: {len(curve['xaxis'])} rows × 2 columns")
    else:
        print("No curve data to export")

## Summary

In this notebook, you learned:
1. ✓ How to query and retrieve result data from simulation tools
2. ✓ How to use `.select()` to specify which fields to return (required!)
3. ✓ How to download specific output fields using the `download()` method
4. ✓ How to access curve and scalar data from results
5. ✓ How to save data in JSON format
6. ✓ How to export curve data to CSV for analysis

## Key Points

- **Always use `.select()`**: The API requires you to specify which fields to return before calling `.execute()`
- **Use JSON for complete data**: Preserves all structure and metadata
- **Use CSV for curve data**: Better for plotting and analysis tools
- **Include metadata**: Always save SQUID and input parameters with results
- **Download method**: Use `download(tool, squid, field=...)` to get specific output fields

## API Methods for Data Access

1. **`query().select().execute()`**: Search for results with specific fields
   - Requires `.select()` to specify return fields
   - Use `.filter()` to add search conditions
   
2. **`download(tool, squid, field=...)`**: Download specific output fields
   - Use `field` parameter for specific output fields
   - Returns the data in appropriate format (dict, list, etc.)

## Next Steps

- `04_pagination.ipynb` - Learn how to efficiently iterate over large result sets

In [ ]:
import csv

if sample_result:
    # Find first curve field
    curve_fields = [k for k, v in sample_result.items() 
                    if isinstance(v, dict) and 'xaxis' in v]
    
    if curve_fields:
        field_name = curve_fields[0]
        curve = sample_result[field_name]
        csv_file = f"st4pnjunction_{field_name.replace('.', '_')}.csv"
        
        with open(csv_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['X', 'Y'])  # Header
            
            # Write data rows
            for x, y in zip(curve['xaxis'], curve['yaxis']):
                writer.writerow([x, y])
        
        print(f"✓ Curve data saved to CSV: {csv_file}")
        print(f"  Format: {len(curve['xaxis'])} rows × 2 columns")
    else:
        print("No curve data to export")


## Summary

In this notebook, you learned:
1. ✓ How to list available files for a SQUID using `get_squid_files()`
2. ✓ How to download files using the `download()` method
3. ✓ How to access curve and scalar data from results
4. ✓ How to save data in JSON format
5. ✓ How to export curve data to CSV for analysis

## Best Practices

- **Check files first**: Use `get_squid_files()` to see what's available before attempting downloads
- **Use JSON for complete data**: Preserves all structure and metadata
- **Use CSV for curve data**: Better for plotting and analysis tools
- **Include metadata**: Always save SQUID and input parameters with results
- **Simulation tools only**: File listing and download only work for simulation tools (simtool=True)

## API Methods for File Access

The library provides two complementary methods:

1. **`get_squid_files(squid, simtool=True)`**: List all files associated with a simulation run
   - Returns file IDs, names, and sizes
   - Use this to discover what's available
   - Only works for simulation tools

2. **`download(tool, squid, field=None, file_name=None, simtool=False)`**: Download specific files or data
   - Use `field` for specific output fields
   - Use `file_name` for files discovered via `get_squid_files()`
   - Set `simtool=True` when downloading from simulation tools

## Next Steps

- `04_pagination.ipynb` - Learn how to efficiently iterate over large result sets